In [ ]:
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os
import numpy as npP
import joblib

In [178]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from statsmodels.tsa.vector_ar.var_model import VAR
from prophet import Prophet

In [172]:
X_columns = [
    'resting_heart_rate',
    'avg_overall_sleep_score',
    'fatigue',
    'mood',
    'stress',
    'sleep_quality',
    'very_active_minutes_sum',
    'sedentary_minutes_sum',
    'resting_heart_rate_14_mean',
    'resting_heart_rate_14_std',
    'avg_overall_sleep_score_14_mean',
    'avg_overall_sleep_score_14_std',
]

y_column = 'Is_High_Risk_Next_7_Days'

In [142]:
df = pd.read_csv("./cleaned_data/p07_model_ready.csv")
df.set_index("dateTime", inplace=True)
df.sort_index(inplace=True)
assert df.isna().sum().sum() == 0, "Data contains NaN values"
df.head()

,resting_heart_rate,avg_overall_sleep_score,fatigue,mood,stress,sleep_quality,very_active_minutes_sum,sedentary_minutes_sum,resting_heart_rate_14_mean,resting_heart_rate_14_std,avg_overall_sleep_score_14_mean,avg_overall_sleep_score_14_std,Subjective_Distress_Score,Physiological_Deviation_Score,Composite_Risk_Score,High_Risk_State,Is_High_Risk_Next_7_Days
dateTime,,,,,,,,,,,,,,,,,
2019-11-21,55.481817,84.0,2.0,3.0,4.0,3.0,94,517,54.523219,1.594162,78.428571,7.977992,1,0,1,0,0
2019-11-22,57.737835,67.0,3.0,3.0,3.0,2.0,9,830,54.804446,1.791905,77.357143,8.454468,1,2,3,1,0
2019-11-23,56.930720,85.0,3.0,3.0,4.0,4.0,30,623,55.039024,1.842910,78.000000,8.682431,0,1,1,0,1
2019-11-24,59.165880,48.0,3.0,3.0,4.0,3.0,86,855,55.423071,2.104138,76.000000,11.832160,0,2,2,0,1
2019-11-25,56.551208,89.0,3.0,3.0,4.0,4.0,49,537,55.704120,1.958144,76.214286,12.052313,0,0,0,0,1


In [143]:
X = df[X_columns]
y = df[y_column]

In [144]:
def train_test_split(X, y, test_size=0.2):
    # Split the data into training and testing sets
    split_index = int(len(X) * (1 - test_size))
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    X_train.index = pd.to_datetime(X_train.index, errors='coerce')
    X_test.index = pd.to_datetime(X_test.index, errors='coerce')    

    y_train.index = pd.to_datetime(y_train.index, errors='coerce')
    y_test.index = pd.to_datetime(y_test.index, errors='coerce')
    return X_train, X_test, y_train, y_test

In [145]:
def fit_arima_model(X, y, order=(1, 0, 0)):
    model = ARIMA(y, exog=X, order=order)
    model_fit = model.fit()
    return model_fit

def make_predictions(model_fit, X):
    predictions = model_fit.predict(start=len(X) - len(X), end=len(X) - 1, exogenous=X)
    return predictions

def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    r2_score_value = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)   
    return mse, r2_score_value, mae

## Model Definition

models = [
    {
        "name": "SARIMAX",
        "call": SARIMAX,
        "params": {
            "order": (1, 1, 1),
            "seasonal_order": (1, 1, 1, 24)
        }
    },
    {
        "name": "LinearRegression",
        "call": LinearRegression,
        "params": {}
    },
    {
        "name": "XGBoost",
        "call": XGBRegressor,
        "params": {
            "n_estimators": 100,
            "max_depth": 3,
            "learning_rate": 0.1,
            "objective": "reg:squarederror"
        }
    },
    {
        "name": "LightGBM",
        "call": LGBMRegressor,
        "params": {
            "n_estimators": 100,
            "max_depth": 3,
            "learning_rate": 0.1
        }
    },
    {
        "name": "RandomForest",
        "call": RandomForestRegressor,
        "params": {
            "n_estimators": 100,
            "max_depth": 5
        }
    },
    {
        "name": "GradientBoosting",
        "call": GradientBoostingRegressor,
        "params": {
            "n_estimators": 100,
            "max_depth": 3,
            "learning_rate": 0.1
        }
    },
    {
        "name": "ARIMA",
        "call": ARIMA,
        "params": {
            "order": (1, 1, 1)
        }
    }
]


In [179]:
classifiers = [
    {
        "name": "LogisticRegression",
        "call": LogisticRegression,
        "params": {
            "max_iter": 1000,
            "solver": "liblinear",
            "C": 1.0,
            "class_weight": "balanced"
        }
    },
    {
        "name": "XGBoost",
        "call": XGBClassifier,      
        "params": {
            "n_estimators": 100,
            "max_depth": 3,
            "learning_rate": 0.1,
            "objective": "binary:logistic",
            "scale_pos_weight": 1,
            "subsample": 0.8,
            "colsample_bytree": 0.8
        }
    },
    {
        "name": "LightGBM",
        "call": LGBMClassifier,
        "params": {
            "n_estimators": 100,
            "max_depth": 3,
            "learning_rate": 0.1,
            "objective": "binary",
            "class_weight": "balanced",
            "boosting_type": "gbdt"
        }
    },
    {
        "name": "RandomForest",
        "call": RandomForestClassifier,
        "params": {
            "n_estimators": 100,
            "max_depth": 5,
            "class_weight": "balanced",
            "min_samples_split": 5,
            "min_samples_leaf": 2
        }
    },
    {
        "name": "GradientBoosting",
        "call": GradientBoostingClassifier,
        "params": {
            "n_estimators": 100,
            "max_depth": 3,
            "learning_rate": 0.1,
            "subsample": 0.8,
            "min_samples_split": 5
        }
    },
    {
        "name": "SVC",
        "call": SVC,
        "params": {
            "kernel": 'linear',
            "C": 1.0,
            "class_weight": "balanced",
            "probability": True
        }
    },
    {
        "name": "DecisionTree",
        "call": DecisionTreeClassifier,
        "params": {
            "max_depth": 5,
            "class_weight": "balanced",
            "min_samples_split": 5,
            "min_samples_leaf": 2
        }
    },
    {
        "name": "KNeighbors",
        "call": KNeighborsClassifier,
        "params": {
            "n_neighbors": 5,
            "weights": "distance",
            "metric": "minkowski"
        }
    },
    {
        "name": "NaiveBayes",
        "call": GaussianNB,
        "params": {
            "var_smoothing": 1e-9
        }
    }
]

time_series_models = [
    {
        "name": "SARIMAX",
        "call": SARIMAX,
        "params": {
            "order": (1, 1, 1),
            "seasonal_order": (1, 1, 1, 7),  # Weekly seasonality
            "enforce_stationarity": False,
            "enforce_invertibility": False
        }
    },
    {
        "name": "SARIMAX_Daily",
        "call": SARIMAX,
        "params": {
            "order": (2, 1, 2),
            "seasonal_order": (1, 1, 1, 1),  # Daily seasonality
            "enforce_stationarity": False,
            "enforce_invertibility": False
        }
    },
    {
        "name": "ARIMA_Simple",
        "call": ARIMA,
        "params": {
            "order": (1, 1, 1),
            "trend": 'c'
        }
    },
    {
        "name": "ARIMA_Complex",
        "call": ARIMA,
        "params": {
            "order": (2, 1, 2),
            "trend": 'c'
        }
    },
    
    {
        "name": "Prophet",
        "call": Prophet,
        "params": {
            "yearly_seasonality": True,
            "weekly_seasonality": True,
            "daily_seasonality": True,
            "changepoint_prior_scale": 0.05
        }
    },
    {
        "name": "VAR",
        "call": VAR,
        "params": {
            "maxlags": 15,
            "ic": 'aic'
        }
    },
    {
        "name": "SARIMA_Weekly",
        "call": SARIMAX,
        "params": {
            "order": (1, 0, 1),
            "seasonal_order": (1, 1, 1, 7),
            "trend": 'c'
        }
    },
    {
        "name": "SARIMA_Monthly",
        "call": SARIMAX,
        "params": {
            "order": (1, 0, 1),
            "seasonal_order": (1, 1, 1, 30),
            "trend": 'c'
        }
    }
]

In [184]:
def train_models(models, X_train, y_train):
    """
    Trains multiple time series models on the provided training data.
    Args:
        models (list): List of dictionaries containing model names, calls, and parameters.
        X_train (pd.DataFrame or np.ndarray): Training features (exogenous variables).
        y_train (pd.Series or np.ndarray): Training target series.
    Returns:
        dict: Dictionary containing trained models.
    """

    trained_models = {}
    if not isinstance(models, list):
        raise ValueError("The 'models' argument must be a list of dictionaries containing model information.")
    if len(y_train) == 0:
        raise ValueError("The training data is empty. Please provide valid data for training.")

    for model in models:
        model_name = model["name"]
        model_call = model["call"]
        model_params = model["params"]
        print(f"Preparing to train {model_name} with parameters: {model_params}")
        # Ensure the models directory exists
        if not os.path.exists("./models"):
            os.makedirs("./models")
        if model_name in ["SARIMAX", "ARIMA"]:
            # Pass exogenous variables if available
            if X_train is not None and X_train.shape[1] > 0:
                trained_model = model_call(y_train, exog=X_train, **model_params).fit()
            else:
                trained_model = model_call(y_train, **model_params).fit()
            # joblib.dump(trained_model, f"./models/{model_name}_model.pkl")
        else:
            trained_model = model_call(**model_params).fit(X_train, y_train)
            # joblib.dump(trained_model, f"./models/{model_name}_model.pkl")
        
        trained_models[model_name] = trained_model
        print(f"{model_name} trained successfully.\n")
    print("All models trained successfully.")

    return trained_models

def train_ml_models(models, X_train, y_train):
    """
    Trains multiple models on the provided training data.
    Args:
        models (list): List of dictionaries containing model names, calls, and parameters.
        X_train (pd.DataFrame or np.ndarray): Training features (exogenous variables).
        y_train (pd.Series or np.ndarray): Training target series.
    Returns:
        dict: Dictionary containing trained non-time series models.
    """
    
    trained_models = {}
    if not isinstance(models, list):
        raise ValueError("The 'models' argument must be a list of dictionaries containing model information.")
    if len(y_train) == 0:
        raise ValueError("The training data is empty. Please provide valid data for training.")

    for model in models:
        model_name = model["name"]
        model_call = model["call"]
        model_params = model["params"]
        print(f"Preparing to train {model_name} with parameters: {model_params}")
        
        if not os.path.exists("./models"):
            os.makedirs("./models")
        
        trained_model = model_call(**model_params).fit(X_train, y_train)
        #joblib.dump(trained_model, f"./models/{model_name}_model.pkl")
        
        trained_models[model_name] = trained_model
        print(f"{model_name} trained successfully.\n")
    
    print("All models trained successfully.")
    
    return trained_models

def train_time_series_models(models, X_train, y_train):
    """
    Trains multiple time series models on the provided training data.
    """
    trained_time_series_models = {}
    
    for model in models:
        model_name = model["name"]
        model_call = model["call"]
        model_params = model["params"]
        print(f"Preparing to train {model_name} with parameters: {model_params}")
        
        try:
            if model_name in ["SARIMAX", "ARIMA", "SARIMAX_Daily", "ARIMA_Simple", 
                            "ARIMA_Complex", "SARIMA_Weekly", "SARIMA_Monthly"]:
                # For SARIMAX/ARIMA models, pass endog as first argument
                model_instance = model_call(endog=y_train, exog=X_train, **model_params)
                trained_model = model_instance.fit(disp=False)
            
            elif model_name == "VAR":
                # For VAR, combine X and y into one dataframe
                combined_data = pd.concat([y_train, X_train], axis=1)
                model_instance = model_call(combined_data)
                trained_model = model_instance.fit(**model_params)
            
            elif model_name == "Prophet":
                # For Prophet, prepare data in required format
                prophet_data = pd.DataFrame({
                    'ds': y_train.index,
                    'y': y_train.values
                })
                model_instance = model_call(**model_params)
                for column in X_train.columns:
                    model_instance.add_regressor(column)
                trained_model = model_instance.fit(prophet_data)
            
            else:
                trained_model = model_call(**model_params).fit(X_train, y_train)
            
            trained_time_series_models[model_name] = trained_model
            print(f"{model_name} trained successfully.\n")
            
        except Exception as e:
            print(f"Error training {model_name}: {str(e)}\n")
            continue
    
    print("Time series models training complete.")
    return trained_time_series_models

In [181]:
def evaluate_models(trained_models,X_test, y_test,model_type = "classification"):
    """
    Evaluates trained models on the test data and calculates Mean Squared Error (MSE).
    Args:
        trained_models (dict): Dictionary containing trained models.
        X_test (pd.DataFrame): Test features.
        y_test (pd.Series): Test target.
    Returns:
        pd.DataFrame: DataFrame containing model names, parameters, and MSE values.
    """
    results = []
    
    for model_name, model in trained_models.items():
        print(f"Evaluating {model_name}...")
        if model_type in "time_series":
            start = len(model.data.endog)
            end = start + len(y_test) - 1
            predictions = model.predict(start=start, end=end, exog=X_test)
            predictions = predictions[:len(y_test)]  # Ensure predictions match test data length

            if np.isnan(predictions).any():
                print(f"Warning: {model_name} predictions contain NaN values. Skipping evaluation for this model.")
                continue
            if len(predictions) != len(y_test):
                print(f"Warning: {model_name} predictions length does not match test data length. Skipping evaluation for this model.")
                continue
            mse = mean_squared_error(y_test, predictions)
            r2_score_value = r2_score(y_test, predictions)
            mae = mean_absolute_error(y_test, predictions)
            print(f"{model_name} MSE: {mse:.4f}, R^2: {r2_score_value:.4f}, MAE: {mae:.4f}")
            results.append({
                "Model": model_name,
                "Parameters": model.params,
                "MSE": mse,
                "MAE": mae,
                "R^2": r2_score_value, 
            })
        elif model_type == "regresssion":
            predictions = model.predict(X_test)
            if len(predictions) != len(y_test):
                print(f"Warning: {model_name} predictions length does not match test data length. Skipping evaluation for this model.")
                continue
            mae = mean_absolute_error(y_test.values, predictions)
            mse = mean_squared_error(y_test.values, predictions)
            r2_score_value = r2_score(y_test.values, predictions)
            print(f"{model_name} MSE: {mse:.4f}, R^2: {r2_score_value:.4f}, MAE: {mae:.4f}")
            results.append({
                "Model": model_name,
                "Parameters": model.get_params(),
                "MSE": mse,
                "MAE": mae,
                "R^2": r2_score_value,                
            })
        elif model_type == "classification":
            predictions = model.predict(X_test)
            if len(predictions) != len(y_test):
                print(f"Warning: {model_name} predictions length does not match test data length. Skipping evaluation for this model.")
                continue
            accuracy = accuracy_score(y_test, predictions)
            f1 = f1_score(y_test, predictions, average='weighted')
            precision = precision_score(y_test, predictions, average='weighted')
            recall = recall_score(y_test, predictions, average='weighted')
            print(f"{model_name} Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
            results.append({
                "Model": model_name,
                "Parameters": model.get_params(),
                "Accuracy": accuracy,
                "F1 Score": f1,
                "Precision": precision,
                "Recall": recall,
            })
        else:
            print(f"Model {model_name} is not recognized for evaluation.")
    print("Evaluation complete.")
    return pd.DataFrame(results)

## testing

In [192]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
assert X_train.index.dtype == 'datetime64[ns]', "X_train index is not datetime"
assert X_test.index.dtype == 'datetime64[ns]', "X_test index is not datetime"  
assert y_train.index.dtype == 'datetime64[ns]', "y_train index is not datetime"
assert y_test.index.dtype == 'datetime64[ns]', "y_test index is not datetime"

In [185]:
models = train_ml_models(classifiers, X_train, y_train)

Preparing to train LogisticRegression with parameters: {'max_iter': 1000, 'solver': 'liblinear', 'C': 1.0, 'class_weight': 'balanced'}
LogisticRegression trained successfully.

Preparing to train XGBoost with parameters: {'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.1, 'objective': 'binary:logistic', 'scale_pos_weight': 1, 'subsample': 0.8, 'colsample_bytree': 0.8}
XGBoost trained successfully.

Preparing to train LightGBM with parameters: {'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.1, 'objective': 'binary', 'class_weight': 'balanced', 'boosting_type': 'gbdt'}
[LightGBM] [Info] Number of positive: 52, number of negative: 48
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000193 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 263
[LightGBM] [Info] Number of data points in the train set: 100, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -

In [186]:
time_series_models_trained = train_time_series_models(time_series_models, X_train, y_train)

Preparing to train SARIMAX with parameters: {'order': (1, 1, 1), 'seasonal_order': (1, 1, 1, 7), 'enforce_stationarity': False, 'enforce_invertibility': False}


c:\Users\ahmed\miniconda3\envs\ds1\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\ahmed\miniconda3\envs\ds1\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\ahmed\miniconda3\envs\ds1\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\ahmed\miniconda3\envs\ds1\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\ahmed\miniconda3\envs\ds1\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency i

SARIMAX trained successfully.

Preparing to train SARIMAX_Daily with parameters: {'order': (2, 1, 2), 'seasonal_order': (1, 1, 1, 1), 'enforce_stationarity': False, 'enforce_invertibility': False}
Error training SARIMAX_Daily: Seasonal periodicity must be greater than 1.

Preparing to train ARIMA_Simple with parameters: {'order': (1, 1, 1), 'trend': 'c'}
Error training ARIMA_Simple: In models with integration (`d > 0`) or seasonal integration (`D > 0`), trend terms of lower order than `d + D` cannot be (as they would be eliminated due to the differencing operation). For example, a constant cannot be included in an ARIMA(1, 1, 1) model, but including a linear trend, which would have the same effect as fitting a constant to the differenced data, is allowed.

Preparing to train ARIMA_Complex with parameters: {'order': (2, 1, 2), 'trend': 'c'}
Error training ARIMA_Complex: In models with integration (`d > 0`) or seasonal integration (`D > 0`), trend terms of lower order than `d + D` cannot

c:\Users\ahmed\miniconda3\envs\ds1\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\ahmed\miniconda3\envs\ds1\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\ahmed\miniconda3\envs\ds1\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\ahmed\miniconda3\envs\ds1\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\ahmed\miniconda3\envs\ds1\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency i

SARIMA_Weekly trained successfully.

Preparing to train SARIMA_Monthly with parameters: {'order': (1, 0, 1), 'seasonal_order': (1, 1, 1, 30), 'trend': 'c'}


c:\Users\ahmed\miniconda3\envs\ds1\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


SARIMA_Monthly trained successfully.

Time series models training complete.


In [187]:
evaluation_results = evaluate_models(models, X_test, y_test)

Evaluating LogisticRegression...
LogisticRegression Accuracy: 0.4000, F1 Score: 0.4306, Precision: 0.4897, Recall: 0.4000
Evaluating XGBoost...
XGBoost Accuracy: 0.6400, F1 Score: 0.6048, Precision: 0.5843, Recall: 0.6400
Evaluating LightGBM...
LightGBM Accuracy: 0.6400, F1 Score: 0.6310, Precision: 0.6239, Recall: 0.6400
Evaluating RandomForest...
RandomForest Accuracy: 0.6400, F1 Score: 0.5620, Precision: 0.5009, Recall: 0.6400
Evaluating GradientBoosting...
GradientBoosting Accuracy: 0.6400, F1 Score: 0.6048, Precision: 0.5843, Recall: 0.6400
Evaluating SVC...
SVC Accuracy: 0.4400, F1 Score: 0.4672, Precision: 0.5138, Recall: 0.4400
Evaluating DecisionTree...
DecisionTree Accuracy: 0.5600, F1 Score: 0.5169, Precision: 0.4800, Recall: 0.5600
Evaluating KNeighbors...
KNeighbors Accuracy: 0.4800, F1 Score: 0.5065, Precision: 0.5685, Recall: 0.4800
Evaluating NaiveBayes...
NaiveBayes Accuracy: 0.4000, F1 Score: 0.4114, Precision: 0.4235, Recall: 0.4000
Evaluation complete.


In [188]:
evaluation_results

,Model,Parameters,Accuracy,F1 Score,Precision,Recall
0,LogisticRegression,"{'C': 1.0, 'class_weight': 'balanced', 'dual':...",0.40,0.430560,0.489744,0.40
1,XGBoost,"{'objective': 'binary:logistic', 'base_score':...",0.64,0.604755,0.584286,0.64
2,LightGBM,"{'boosting_type': 'gbdt', 'class_weight': 'bal...",0.64,0.631019,0.623860,0.64
3,RandomForest,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.64,0.561951,0.500870,0.64
4,GradientBoosting,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...",0.64,0.604755,0.584286,0.64
5,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.44,0.467222,0.513766,0.44
6,DecisionTree,"{'ccp_alpha': 0.0, 'class_weight': 'balanced',...",0.56,0.516923,0.480000,0.56
7,KNeighbors,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...",0.48,0.506486,0.568462,0.48
8,NaiveBayes,"{'priors': None, 'var_smoothing': 1e-09}",0.40,0.411429,0.423529,0.40


In [189]:
time_series_evaluation = evaluate_models(time_series_models_trained, X_test, y_test, model_type="time_series")

Evaluating SARIMAX...
SARIMAX MSE: 0.6186, R^2: -2.0683, MAE: 0.5561
Evaluating SARIMA_Weekly...
SARIMA_Weekly MSE: 0.4803, R^2: -1.3822, MAE: 0.5034
Evaluating SARIMA_Monthly...
SARIMA_Monthly MSE: 0.3356, R^2: -0.6647, MAE: 0.4062
Evaluation complete.


In [190]:
time_series_evaluation

,Model,Parameters,MSE,MAE,R^2
0,SARIMAX,resting_heart_rate -0.053805 av...,0.618576,0.556129,-2.068332
1,SARIMA_Weekly,intercept -0.030818 re...,0.480250,0.503433,-1.382194
2,SARIMA_Monthly,intercept -0.097576 re...,0.335604,0.406242,-0.664701
